## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [1]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin8_1 = pd.read_csv("../../바이너리_최종데이터_1004/gcc8/o1/o1_bincore8.csv", index_col=0)
print(bin8_1.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin8_1.reset_index(inplace=True, drop=True)

print('shape -', bin8_1.shape)
print('reset_index 완료')
print('input data shape')
bin8_1.head()

(27667301, 2)
shape - (27667301, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [2]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin8_1['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin8_1['label'].value_counts())

256
0    27628838
1       38463
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [3]:
idx_bin = bin8_1[bin8_1['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 32 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin8_1), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin8_1', len(ls_idx_bin))

# loc 로 수정필요
bin8_1_Ngram = bin8_1.loc[ls_idx_bin,:].copy()

1230816
나머지 0
최종 길이 1230816
bin8_1 1230816


## (4) false data 만들기

In [4]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin8_1_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin8_1.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin8_1)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin8_1[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

38463.0
0 150536
1000 17310233
2000 16164413
3000 16751654
4000 25986866
5000 11544382
6000 16837160
7000 15851052
8000 22500926
9000 11656610
10000 20648104
11000 14896371
12000 5220955
13000 20380664
14000 22734594
15000 265437
16000 20236924
17000 2574092
18000 5658087
19000 16673995
20000 141863
21000 24521297
22000 10763987
23000 15068494
24000 21898565
25000 15243658
26000 1019024
27000 24398087
28000 14575330
29000 1066026
30000 5425698
31000 345597
32000 2552784
33000 3016742
34000 12508799
35000 23789287
36000 19771613
37000 6232359
38000 9376716
완료
38463


In [5]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin8_1['label'].value_counts()[1])

38463
38463


## (5) False Data + True Data 합치기

In [6]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin8_1_Ngram])
final.shape

(2461632, 2)

## (6) one hot encoding

In [7]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc8_1_onehot_Ngram = pd.get_dummies(final['bin'])
bc8_1_onehot_Ngram = pd.concat([final['label'], bc8_1_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc8_1_onehot_Ngram.shape)

원핫인코딩완료
(2461632, 257)


In [8]:
# 훈련 데이터, 훈련 라벨
x_bc8_1 = bc8_1_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc8_1 = bc8_1_onehot_Ngram['label'].to_numpy()
print(x_bc8_1.shape, x_bc8_1.shape)

x_bc8_1 = x_bc8_1.reshape(-1, right_idx, x_bc8_1.shape[1])
y_bc8_1 = y_bc8_1.reshape(-1, right_idx, 1)

print(x_bc8_1.shape, y_bc8_1.shape)

(2461632, 256) (2461632, 256)
(76926, 32, 256) (76926, 32, 1)


In [9]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc8_1.shape[0])

x_bc8_1 = x_bc8_1[p]
y_bc8_1 = y_bc8_1[p]

print(x_bc8_1.shape, y_bc8_1.shape)

(76926, 32, 256) (76926, 32, 1)


## (7) 모델

In [10]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(48, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [11]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc8_1, y_bc8_1):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc8_1[train],
               y_bc8_1[train],
               epochs = 10,
               batch_size = 32,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc8_1[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc8_1[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69233 samples
Epoch 1/10
69233/69233 [==============================] - 100s 1ms/sample - loss: 0.0158 - accuracy: 0.9957
Epoch 2/10
69233/69233 [==============================] - 32s 461us/sample - loss: 0.0029 - accuracy: 0.9991
Epoch 3/10
69233/69233 [==============================] - 39s 563us/sample - loss: 0.0018 - accuracy: 0.9994
Epoch 4/1

accuracy_score 0.9997318991290783
recall_score 0.9894823674984533
precision_score 0.9968834406814876
f1_score 0.9931691161250259
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69233 samples
Epoch 1/10
69233/69233 [==============================] - 85s 1ms/sample - loss: 4.6489e-04 - accuracy: 0.9999
Epoch 2/10
69233/69233 [==============================] - 46s 669us/sample - loss: 4.0208e-04

accuracy_score 0.9998618874301313
recall_score 0.9957627118644068
precision_score 0.9970301230377598
f1_score 0.9963960144159424
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69233 samples
Epoch 1/10
69233/69233 [==============================] - 79s 1ms/sample - loss: 2.2357e-04 - accuracy: 0.9999
Epoch 2/10
69233/69233 [==============================] - 45s 652us/sample - loss: 1.5740e-04

accuracy_score 0.999914695177434
recall_score 0.9987824675324676
precision_score 0.9969617176422929
f1_score 0.9978712620375063
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69233 samples
Epoch 1/10
69233/69233 [==============================] - 91s 1ms/sample - loss: 1.0371e-04 - accuracy: 1.0000
Epoch 2/10
69233/69233 [==============================] - 47s 679us/sample - loss: 7.5222e-05 

accuracy_score 0.9999796893279604
recall_score 0.999580800670719
precision_score 0.9993713327745181
f1_score 0.9994760557476684
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69233 samples
Epoch 1/10
69233/69233 [==============================] - 103s 1ms/sample - loss: 4.7820e-05 - accuracy: 1.0000
Epoch 2/10
69233/69233 [==============================] - 45s 656us/sample - loss: 4.1902e-05

...

accuracy_score 0.999995937865592
recall_score 1.0
precision_score 0.9997918834547347
f1_score 0.9998959308981163
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69234 samples
Epoch 1/10
69234/69234 [==============================] - 105s 2ms/sample - loss: 1.8192e-05 - accuracy: 1.0000
Epoch 2/10
69234/69234 [==============================] - 45s 651us/sample - loss: 1.5522e-05 - accuracy: 1.

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69234 samples
Epoch 1/10
69234/69234 [==============================] - 119s 2ms/sample - loss: 1.4480e-05 - accuracy: 1.0000
Epoch 2/10
69234/69234 [==============================] - 46s 670us/sample - loss: 1.3964e-05 - accuracy: 1.0000
Epoch 3/10
69234/69234 [===============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69234 samples
Epoch 1/10
69234/69234 [==============================] - 106s 2ms/sample - loss: 1.3945e-05 - accuracy: 1.0000
Epoch 2/10
69234/69234 [==============================] - 47s 672us/sample - loss: 1.3954e-05 - accuracy: 1.0000
Epoch 3/10
69234/69234 [===============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 69234 samples
Epoch 1/10
69234/69234 [==============================] - 62s 899us/sample - loss: 1.4122e-05 - accuracy: 1.0000
Epoch 2/10
69234/69234 [==============================] - 47s 681us/sample - loss: 1.3930e-05 - accuracy: 1.0000
Epoch 3/10
69234/69234 [==============

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.9997318991290783, 0.9998618874301313, 0.999914695177434, 0.9999796893279604, 0.9999756271935526, 0.999995937865592, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Recall: [0.9894823674984533, 0.9957627118644068, 0.9987824675324676, 0.999580800670719, 0.9993893751272135, 1.0, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [0.9968834406814876, 0.9970301230377598, 0.9969617176422929, 0.9993713327745181, 0.9993893751272135, 0.9997918834547347, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9931691161250259, 0.9963960144159424, 0.9978712620375063, 0.9994760557476684, 0.9993893751272135, 0.9998959308981163, 1.0, 1.0, 1.0, 1.0]


## (9) 평가지표

In [12]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9999459736123748
10-Fold Cross_validation. Recall : 0.9982997722693261
10-Fold Cross_validation. Precision : 0.9989427872718007
10-Fold Cross_validation. F1-Score : 0.9986197754351472


In [13]:
model1.save('gcc8_bin_core_s32_h48_o1.h5')
print('save 완료')

save 완료
